In [3]:
# USAGE
# python opencv_text_detection_image.py --image images/lebron_james.jpg
# --east frozen_east_text_detection.pb

# import the necessary packages
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import time
import cv2
import glob

In [5]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", type=str,
                help="path to input image")
ap.add_argument("-east", "--east", type=str,
                help="path to input EAST text detector")
ap.add_argument("-c", "--min-confidence", type=float, default=0.5,
                help="minimum probability required to inspect a region")
ap.add_argument("-w", "--width", type=int, default=320,
                help="resized image width (should be multiple of 32)")
ap.add_argument("-e", "--height", type=int, default=320,
                help="resized image height (should be multiple of 32)")
args = vars(ap.parse_args())


usage: ipykernel_launcher.py [-h] [-i IMAGE] [-east EAST] [-c MIN_CONFIDENCE]
                             [-w WIDTH] [-e HEIGHT]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Sanjeev\AppData\Roaming\jupyter\runtime\kernel-2095aaf0-f35e-44e3-a58d-e283313efa10.json


SystemExit: 2

In [7]:
img_path ="C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/X00016469612.jpg"
i =cv2.imread(img_path)
i.shape

(1013, 463, 3)

In [30]:
img_path ="C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/X00016469612.jpg"
model_path="C:/Users/Sanjeev/Documents/Python Scripts/NLP/frozen_east_text_detection.pb"
min_confidence=0.2

In [31]:
# load the input image and grab the image dimensions
# image = cv2.imread(args["image"])
image = cv2.imread(img_path)      # image path 
orig = image.copy()
(H, W) = image.shape[:2]

# set the new width and height and then determine the ratio in change
# for both the width and height
# (newW, newH) = (args["width"], args["height"])
(newW, newH) = 320,320
rW = W / float(newW)
rH = H / float(newH)

# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
# net = cv2.dnn.readNet(args["east"])
net = cv2.dnn.readNet(model_path) 

[INFO] loading EAST text detector...


In [32]:
# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
                             (123.68, 116.78, 103.94), swapRB=True, crop=False)
start = time.time()
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)
end = time.time()

# show timing information on text prediction
print("[INFO] text detection took {:.6f} seconds".format(end - start))

[INFO] text detection took 0.748536 seconds


In [33]:
len(geometry[0,0,1]),geometry[0,0,1]

(80,
 array([128.69878 ,  72.914734,  78.29735 ,  82.27043 ,  88.61762 ,
         96.60928 , 107.04452 ,  97.37418 ,  72.91046 ,  51.019302,
         35.837692,  26.807367,  20.058367,  15.56344 ,  13.813998,
         13.061317,  12.029698,  12.187435,  12.406771,  12.627942,
         13.012527,  14.540729,  13.399692,  11.521936,  10.194643,
          9.3542  ,   9.141606,   8.81364 ,   8.364239,   8.60757 ,
          8.716683,   8.920558,   9.271866,   9.398319,   9.558159,
          9.757291,  10.050643,  10.393915,  11.053546,  12.097688,
         11.802659,  10.498992,  10.151093,   9.270392,   8.921176,
          9.092626,   8.615006,   8.75773 ,   8.919279,   8.870549,
          9.192727,   9.608158,   9.898827,  10.625705,  11.500729,
         13.137874,  14.876928,  16.700249,  19.926817,  22.886831,
         27.89952 ,  34.321037,  42.026043,  50.657825,  57.97627 ,
         61.8701  ,  63.112904,  63.298267,  62.19    ,  60.285286,
         56.797215,  53.303562,  50.01523 ,

In [34]:
# grab the number of rows and columns from the scores volume, then
# initialize our set of bounding box rectangles and corresponding
# confidence scores
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []

# loop over the number of rows
for y in range(0, numRows):
    # extract the scores (probabilities), followed by the geometrical
    # data used to derive potential bounding box coordinates that
    # surround text
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]

    # loop over the number of columns
    for x in range(0, numCols):
        # if our score does not have sufficient probability, ignore it
#         if scoresData[x] < args["min_confidence"]:
        if scoresData[x] < min_confidence:
            continue

        # compute the offset factor as our resulting feature maps will
        # be 4x smaller than the input image
        (offsetX, offsetY) = (x * 4.0, y * 4.0)

        # extract the rotation angle for the prediction and then
        # compute the sin and cosine
        angle = anglesData[x]
        cos = np.cos(angle)
        sin = np.sin(angle)

        # use the geometry volume to derive the width and height of
        # the bounding box
        h = xData0[x] + xData2[x]
        w = xData1[x] + xData3[x]

        # compute both the starting and ending (x, y)-coordinates for
        # the text prediction bounding box
        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
        startX = int(endX - w)
        startY = int(endY - h)

        # add the bounding box coordinates and probability score to
        # our respective lists
        rects.append((startX, startY, endX, endY))
        confidences.append(scoresData[x])

# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
boxes = non_max_suppression(np.array(rects), probs=confidences)

# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
    # scale the bounding box coordinates based on the respective
    # ratios
    startX = int(startX * rW)
    startY = int(startY * rH)
    endX = int(endX * rW)
    endY = int(endY * rH)

    # draw the bounding box on the image
    cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)

# show the output image
cv2.imshow("Text Detection", orig)
cv2.waitKey(0)

-1